# Simple 2 Layer CNN on TensorFlow with ReLU Activation

This CNN is a 7 layer 
http://yeephycho.github.io/2016/08/15/image-data-in-tensorflow/
https://github.com/yeephycho/tensorflow_input_image_by_tfrecord/blob/master/src/build_image_data.py

This CNN is a 2 Layer convnet running on TensorFlow and using ReLu activation, while also importing the dataset from a local folder. The code was a modified version of the one found here: https://raw.githubusercontent.com/aymericdamien/TensorFlow-Examples/master/examples/5_DataManagement/build_an_image_dataset.py

In [1]:
import tensorflow as tf # tensorflow module
import numpy as np # numpy module
import os # path join

# disables warning for my CPU
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
config = tf.ConfigProto()
config.intra_op_parallelism_threads = 44
config.inter_op_parallelism_threads = 44

DATA_DIR = "C:/Users/chris/Documents/CUNY/DATA698 - Final Masters Thesis/Images/"
TRAINING_SET_SIZE = 100
BATCH_SIZE = 100
IMAGE_SIZE = 224

In [2]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

# image object from protobuf
class _image_object:
    def __init__(self):
        self.image = tf.Variable([], dtype = tf.string)
        self.height = tf.Variable([], dtype = tf.int64)
        self.width = tf.Variable([], dtype = tf.int64)
        self.filename = tf.Variable([], dtype = tf.string)
        self.label = tf.Variable([], dtype = tf.int32)

def read_and_decode(filename_queue):
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(serialized_example, features = {
        "image/encoded": tf.FixedLenFeature([], tf.string),
        "image/height": tf.FixedLenFeature([], tf.int64),
        "image/width": tf.FixedLenFeature([], tf.int64),
        "image/filename": tf.FixedLenFeature([], tf.string),
        "image/class/label": tf.FixedLenFeature([], tf.int64),})
    image_encoded = features["image/encoded"]
    image_raw = tf.image.decode_jpeg(image_encoded, channels=3)
    image_object = _image_object()
    image_object.image = tf.image.resize_image_with_crop_or_pad(image_raw, IMAGE_SIZE, IMAGE_SIZE)
    image_object.height = features["image/height"]
    image_object.width = features["image/width"]
    image_object.filename = features["image/filename"]
    image_object.label = tf.cast(features["image/class/label"], tf.int64)
    return image_object

In [3]:
def flower_input(if_random = True, if_training = True):
    filenames = [os.path.join(DATA_DIR, "train-0000%d-of-00002" % i) for i in range(0, 1)]

    for f in filenames:
        if not tf.gfile.Exists(f):
            raise ValueError("Failed to find file: " + f)
    filename_queue = tf.train.string_input_producer(filenames)
    image_object = read_and_decode(filename_queue)
    image = tf.image.per_image_standardization(image_object.image)
#    image = image_object.image
    image = tf.image.adjust_gamma(tf.cast(image_object.image, tf.float32), gamma=1, gain=1) # Scale image to (0, 1)
    label = image_object.label
    filename = image_object.filename

    if(if_random):
        min_fraction_of_examples_in_queue = 0.4
        min_queue_examples = int(TRAINING_SET_SIZE * min_fraction_of_examples_in_queue)
        print("Filling queue with %d images before starting to train. " "This will take a few minutes." % min_queue_examples)
        num_preprocess_threads = 1
        image_batch, label_batch, filename_batch = tf.train.shuffle_batch(
            [image, label, filename],
            batch_size = BATCH_SIZE,
            num_threads = num_preprocess_threads,
            capacity = min_queue_examples + 3 * BATCH_SIZE,
            min_after_dequeue = min_queue_examples)
        return image_batch, label_batch, filename_batch
    else:
        image_batch, label_batch, filename_batch = tf.train.batch(
            [image, label, filename],
            batch_size = BATCH_SIZE,
            num_threads = 1)
        return image_batch, label_batch, filename_batch

In [4]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.05)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.02, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

def flower_inference(image_batch):
    W_conv1 = weight_variable([5, 5, 3, 32])
    b_conv1 = bias_variable([32])

    x_image = tf.reshape(image_batch, [-1, IMAGE_SIZE, IMAGE_SIZE, 3])

    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1) # 112

    W_conv2 = weight_variable([5, 5, 32, 64])
    b_conv2 = bias_variable([64])

    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
    h_pool2 = max_pool_2x2(h_conv2) # 56

    W_conv3 = weight_variable([5, 5, 64, 128])
    b_conv3 = bias_variable([128])

    h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
    h_pool3 = max_pool_2x2(h_conv3) # 28

    W_conv4 = weight_variable([5, 5, 128, 256])
    b_conv4 = bias_variable([256])

    h_conv4 = tf.nn.relu(conv2d(h_pool3, W_conv4) + b_conv4)
    h_pool4 = max_pool_2x2(h_conv4) # 14

    W_conv5 = weight_variable([5, 5, 256, 256])
    b_conv5 = bias_variable([256])

    h_conv5 = tf.nn.relu(conv2d(h_pool4, W_conv5) + b_conv5)
    h_pool5 = max_pool_2x2(h_conv5) # 7

    W_fc1 = weight_variable([7*7*256, 2048])
    b_fc1 = bias_variable([2048])

    h_pool5_flat = tf.reshape(h_pool5, [-1, 7*7*256])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool5_flat, W_fc1) + b_fc1)

    h_fc1_drop = tf.nn.dropout(h_fc1, 1.0)

    W_fc2 = weight_variable([2048, 256])
    b_fc2 = bias_variable([256])

    h_fc2 = tf.nn.relu(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

    W_fc3 = weight_variable([256, 64])
    b_fc3 = bias_variable([64])

    h_fc3 = tf.nn.relu(tf.matmul(h_fc2, W_fc3) + b_fc3)

    W_fc4 = weight_variable([64, 5])
    b_fc4 = bias_variable([5])

    y_conv = tf.nn.softmax(tf.matmul(h_fc3, W_fc4) + b_fc4)
#    y_conv = tf.matmul(h_fc3, W_fc4) + b_fc4

    return y_conv

In [5]:
def flower_train():
    image_batch_out, label_batch_out, filename_batch = flower_input(if_random = False, if_training = True)

    image_batch_placeholder = tf.placeholder(tf.float32, shape=[BATCH_SIZE, 224, 224, 3])
    image_batch = tf.reshape(image_batch_out, (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, 3))

    label_batch_placeholder = tf.placeholder(tf.float32, shape=[BATCH_SIZE, 5])
    label_offset = -tf.ones([BATCH_SIZE], dtype=tf.int64, name="label_batch_offset")
    label_batch_one_hot = tf.one_hot(tf.add(label_batch_out, label_offset), depth=5, on_value=1.0, off_value=0.0)

    logits_out = flower_inference(image_batch_placeholder)
#    loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits(labels=label_batch_one_hot, logits=logits_out))
    loss = tf.losses.mean_squared_error(labels=label_batch_placeholder, predictions=logits_out)

    train_step = tf.train.GradientDescentOptimizer(0.0005).minimize(loss)

    saver = tf.train.Saver()

    with tf.Session() as sess:
        # Visualize the graph through tensorboard.
        file_writer = tf.summary.FileWriter("./logs", sess.graph)

        sess.run(tf.global_variables_initializer())
        # saver.restore(sess, "C:/Users/chris/Documents/CUNY/DATA698 - Final Masters Thesis/Images/checkpoint-train.ckpt")
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord, sess = sess)

        for i in range(TRAINING_SET_SIZE):
            image_out, label_out, label_batch_one_hot_out, filename_out = sess.run([image_batch, label_batch_out, label_batch_one_hot, filename_batch])

            _, infer_out, loss_out = sess.run([train_step, logits_out, loss], feed_dict={image_batch_placeholder: image_out, label_batch_placeholder: label_batch_one_hot_out})

            print(i)
            print(image_out.shape)
            print("label_out: ")
            # print(filename_out)
            print(label_out)
            # print(label_batch_one_hot_out)
            # print("infer_out: ")
            # print(infer_out)
            print("loss: ")
            print(loss_out)
            # if(i%50 == 0):
            #    saver.save(sess, "C:/Users/chris/Documents/CUNY/DATA698 - Final Masters Thesis/Images/checkpoint-train.ckpt")

        coord.request_stop()
        coord.join(threads)
        sess.close()

In [6]:
flower_train()

0
(100, 224, 224, 3)
label_out: 
[5 7 3 2 7 5 7 4 6 2 5 7 3 2 7 5 7 4 6 2 5 7 3 2 7 5 7 4 6 2 5 7 3 2 7 5 7
 4 6 2 5 7 3 2 7 5 7 4 6 2 5 7 3 2 7 5 7 4 6 2 5 7 3 2 7 5 7 4 6 2 5 7 3 2
 7 5 7 4 6 2 5 7 3 2 7 5 7 4 6 2 5 7 3 2 7 5 7 4 6 2]
loss: 
0.28
1
(100, 224, 224, 3)
label_out: 
[5 7 3 2 7 5 7 4 6 2 5 7 3 2 7 5 7 4 6 2 5 7 3 2 7 5 7 4 6 2 5 7 3 2 7 5 7
 4 6 2 5 7 3 2 7 5 7 4 6 2 5 7 3 2 7 5 7 4 6 2 5 7 3 2 7 5 7 4 6 2 5 7 3 2
 7 5 7 4 6 2 5 7 3 2 7 5 7 4 6 2 5 7 3 2 7 5 7 4 6 2]
loss: 
0.28
2
(100, 224, 224, 3)
label_out: 
[5 7 3 2 7 5 7 4 6 2 5 7 3 2 7 5 7 4 6 2 5 7 3 2 7 5 7 4 6 2 5 7 3 2 7 5 7
 4 6 2 5 7 3 2 7 5 7 4 6 2 5 7 3 2 7 5 7 4 6 2 5 7 3 2 7 5 7 4 6 2 5 7 3 2
 7 5 7 4 6 2 5 7 3 2 7 5 7 4 6 2 5 7 3 2 7 5 7 4 6 2]
loss: 
0.28
3
(100, 224, 224, 3)
label_out: 
[5 7 3 2 7 5 7 4 6 2 5 7 3 2 7 5 7 4 6 2 5 7 3 2 7 5 7 4 6 2 5 7 3 2 7 5 7
 4 6 2 5 7 3 2 7 5 7 4 6 2 5 7 3 2 7 5 7 4 6 2 5 7 3 2 7 5 7 4 6 2 5 7 3 2
 7 5 7 4 6 2 5 7 3 2 7 5 7 4 6 2 5 7 3 2 7 5 7 4 6 2]
loss: 
0.28
4
(1

In [7]:
def flower_eval():
    image_batch_out, label_batch_out, filename_batch = flower_input(if_random = False, if_training = False)

    image_batch_placeholder = tf.placeholder(tf.float32, shape=[BATCH_SIZE, 224, 224, 3])
    image_batch = tf.reshape(image_batch_out, (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, 3))

    label_tensor_placeholder = tf.placeholder(tf.int64, shape=[BATCH_SIZE])
    label_offset = -tf.ones([BATCH_SIZE], dtype=tf.int64, name="label_batch_offset")
    label_batch = tf.add(label_batch_out, label_offset)

    logits_out = tf.reshape(flower_inference(image_batch_placeholder), [BATCH_SIZE, 5])
    logits_batch = tf.to_int64(tf.argmax(logits_out, dimension = 1))

    correct_prediction = tf.equal(logits_batch, label_tensor_placeholder)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    saver = tf.train.Saver()

    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        # saver.restore(sess, "C:/Users/chris/Documents/CUNY/DATA698 - Final Masters Thesis/Images/checkpoint-train.ckpt")
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord, sess = sess)

        accuracy_accu = 0

        for i in range(29):
            image_out, label_out, filename_out = sess.run([image_batch, label_batch, filename_batch])

            accuracy_out, logits_batch_out = sess.run([accuracy, logits_batch], feed_dict={image_batch_placeholder: image_out, label_tensor_placeholder: label_out})
            accuracy_accu += accuracy_out

            print(i)
            print(image_out.shape)
            print("label_out: ")
            # print(filename_out)
            print(label_out)
            print(logits_batch_out)

        print("Accuracy: ")
        print(accuracy_accu / 29)

        coord.request_stop()
        coord.join(threads)
        sess.close()

In [8]:
#flower_train()
flower_eval()

Instructions for updating:
Use `argmax` instead
0
(100, 224, 224, 3)
label_out: 
[4 6 2 1 6 4 6 3 5 1 4 6 2 1 6 4 6 3 5 1 4 6 2 1 6 4 6 3 5 1 4 6 2 1 6 4 6
 3 5 1 4 6 2 1 6 4 6 3 5 1 4 6 2 1 6 4 6 3 5 1 4 6 2 1 6 4 6 3 5 1 4 6 2 1
 6 4 6 3 5 1 4 6 2 1 6 4 6 3 5 1 4 6 2 1 6 4 6 3 5 1]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
1
(100, 224, 224, 3)
label_out: 
[4 6 2 1 6 4 6 3 5 1 4 6 2 1 6 4 6 3 5 1 4 6 2 1 6 4 6 3 5 1 4 6 2 1 6 4 6
 3 5 1 4 6 2 1 6 4 6 3 5 1 4 6 2 1 6 4 6 3 5 1 4 6 2 1 6 4 6 3 5 1 4 6 2 1
 6 4 6 3 5 1 4 6 2 1 6 4 6 3 5 1 4 6 2 1 6 4 6 3 5 1]
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
2
(100, 224, 224, 3)
label_out: 
[4 6 2 1 6 4 6 3 5 1 4 6 2 1 6 4 6 3 